# Prague Airbnb - What does the market look like?

**Goal:** Understand the dataset structure and identify patterns worth investigating deeper.

 - who are the hosts
 - where do they list
 - what is the listing health
 - how do they measure success
 - how is the customer decision supported (platform's features)

**Time budget:** approx. 40 minutes

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./../data/raw/listings.csv')
df.columns = df.columns.str.lower()

## 1. Quick Data Overview & Cleaning

In [ ]:
key_cols = ['price', 'neighbourhood_cleansed', 'room_type', 'accommodates', 
            'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating',
            'host_id', 'calculated_host_listings_count', 'host_is_superhost',
            'instant_bookable', 'availability_365', 'minimum_nights', 'reviews_per_month']

missing = df[key_cols].isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
print("Missing values in key columns:")
print(missing_pct[missing_pct > 0].sort_values(ascending=False))

In [ ]:
df['price_clean'] = df['price'].replace(r'[\$,]', '', regex=True).astype(float)

print("PRICE DISTRIBUTION (CZK):")
print(df['price_clean'].describe().round(0))

## 2. Who Are the Hosts?

Is this a fragmented market of small hosts? Are there big professional players?

In [ ]:
host_counts = df.groupby('host_id').size().reset_index(name='listings')
host_counts['host_type'] = pd.cut(host_counts['listings'], 
                                   bins=[0,1,3,10,500], 
                                   labels=['1 listing', '2-3', '4-10', '11+'])

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Listings per Host', 'Share of Total Supply'),
                    specs=[[{"type": "histogram"}, {"type": "pie"}]])

fig.add_trace(
    go.Histogram(x=host_counts['listings'].clip(upper=20), nbinsx=20, 
                 marker_color='#636EFA', name='Hosts'),
    row=1, col=1
)

supply_share = host_counts.groupby('host_type')['listings'].sum()
fig.add_trace(
    go.Pie(labels=supply_share.index, values=supply_share.values, 
           marker_colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA']),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, 
                  title_text="Most hosts have 1 listing, but multi-listing hosts control most supply")
fig.update_xaxes(title_text="Listings per Host (capped at 20)", row=1, col=1)
fig.update_yaxes(title_text="Number of Hosts", row=1, col=1)
fig.show()

**Observation:** Most hosts have 1 listing, but a small group controls disproportionate supply.

**Hypothesis:** Professional vs casual hosts behave differently - need segmented analysis.

## 3. Geographic Distribution of Supply and Price

In [ ]:
district_stats = df.groupby('neighbourhood_cleansed').agg({
    'id': 'count',
    'price_clean': 'median'
}).rename(columns={'id': 'listings'}).sort_values('listings', ascending=False).head(10)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Listings by District', 'Median Price by District'))

fig.add_trace(
    go.Bar(y=district_stats.index, x=district_stats['listings'], orientation='h',
           marker_color='#636EFA', name='Listings'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=district_stats.index, x=district_stats['price_clean'], orientation='h',
           marker_color='#00CC96', name='Price'),
    row=1, col=2
)

fig.update_layout(height=450, showlegend=False, title_text="Praha 1 dominates supply AND price")
fig.update_xaxes(title_text="Number of Listings", row=1, col=1)
fig.update_xaxes(title_text="Median Price (CZK)", row=1, col=2)
fig.update_yaxes(autorange="reversed", row=1, col=1)
fig.update_yaxes(autorange="reversed", row=1, col=2)
fig.show()

Are prime locations visibly in Prague 1?

In [ ]:
sample = df.sample(min(2000, len(df)), random_state=42)

fig = px.scatter_mapbox(sample, 
                        lat='latitude', lon='longitude',
                        color='price_clean',
                        color_continuous_scale='Viridis',
                        range_color=[0, 5000],
                        zoom=11,
                        hover_data=['neighbourhood_cleansed', 'room_type', 'price_clean'],
                        title='Listing Locations (colored by price)')

fig.update_layout(mapbox_style='carto-positron', height=500)
fig.show()

**Observation:** Praha 1 = 37% of supply + highest prices.

**Hypothesis:** Praha 1 is a separate market with different dynamics compared to outer regions.

## 4. Are All Listings Actually Active?

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Reviews in Last 12 Months', 'Availability (next 365 days)'))

fig.add_trace(
    go.Histogram(x=df['number_of_reviews_ltm'].clip(upper=50), nbinsx=30,
                 marker_color='#636EFA'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=df['availability_365'], nbinsx=30,
                 marker_color='#00CC96'),
    row=1, col=2
)

fig.update_layout(height=350, showlegend=False, 
                  title_text="Many listings appear inactive")
fig.show()

zero_reviews = (df['number_of_reviews_ltm'] == 0).sum()
blocked = (df['availability_365'] == 0).sum()
print(f"0 reviews in 12 months: {zero_reviews:} ({zero_reviews/len(df)*100:.0f}%)")
print(f"Completely blocked availability: {blocked:} ({blocked/len(df)*100:.0f}%)")

Are they concentrated anywhere?

In [ ]:
df['is_dead'] = (df['number_of_reviews_ltm'] == 0) | (df['availability_365'] == 0)
df['listing_status'] = df['is_dead'].map({True: 'Dead', False: 'Alive'})

sample = df.sample(min(2000, len(df)), random_state=42)

fig = px.scatter_mapbox(sample, 
                        lat='latitude', lon='longitude',
                        color='listing_status',
                        color_discrete_map={'Alive': '#00CC96', 'Dead': '#EF553B'},
                        zoom=11,
                        hover_data=['neighbourhood_cleansed', 'room_type', 'price_clean', 'number_of_reviews_ltm', 'availability_365'],
                        title='Listing Locations (Dead vs Alive)')

fig.update_layout(mapbox_style='carto-positron', height=500)
fig.show()

dead_count = df['is_dead'].sum()
print(f"Dead listings (either no reviews, or no availability): {dead_count} ({dead_count/len(df)*100:.1f}%)")
print(f"Alive listings: {len(df) - dead_count} ({(len(df) - dead_count)/len(df)*100:.1f}%)")

**Observation:** approx. 23% have zero recent reviews, approx. 25% listings have availability blocked.

**Hypothesis:** Significant supply is dead, regardless of neigbourhood. True active market is smaller.

## 5. Rating Distribution

In [ ]:
ratings = df['review_scores_rating'].dropna()

fig = px.histogram(ratings, nbins=40, title='Rating Distribution - Clustering at 4.5+')
fig.add_vline(x=4.5, line_dash="dash", line_color="red", annotation_text="4.5")
fig.update_layout(height=350, showlegend=False)
fig.show()

print(f"Mean: {ratings.mean():.2f} | 4.5+ stars: {(ratings >= 4.5).mean()*100:.0f}%")

What other metric signifies quality?

In [ ]:
df['superhost'] = df['host_is_superhost'].map({'t': 'Superhost', 'f': 'Non-Superhost'})

fig = make_subplots(rows=1, cols=2, subplot_titles=('Price', 'Reviews LTM'))

for sh_status in ['Superhost', 'Non-Superhost']:
    subset = df[df['superhost'] == sh_status]
    color = '#00CC96' if sh_status == 'Superhost' else '#636EFA'
    
    fig.add_trace(
        go.Box(y=subset['price_clean'].clip(upper=8000), name=sh_status, 
               marker_color=color, boxmean=True),
        row=1, col=1
    )
    fig.add_trace(
        go.Box(y=subset['number_of_reviews_ltm'].clip(upper=100), name=sh_status,
               marker_color=color, boxmean=True, showlegend=False),
        row=1, col=2
    )

fig.update_layout(height=400, title_text="Superhosts: Higher prices AND more bookings")
fig.show()

sh = df[df['host_is_superhost'] == 't']
non_sh = df[df['host_is_superhost'] == 'f']
print(f"Price premium: +{(sh['price_clean'].median()/non_sh['price_clean'].median()-1)*100:.0f}%")
print(f"Booking premium: +{(sh['number_of_reviews_ltm'].mean()/non_sh['number_of_reviews_ltm'].mean()-1)*100:.0f}%")

**Observation:** 85% are 4.5+. Ratings don't differentiate.

**Hypothesis:** Need alternative quality signals. Superhost designation actually works (4.8+ rating, response time 90%+ within 24h, cancellation rate <1%, 10+ stays / 100 nights on 3+ stays).

## 6. Minimum Nights

In [ ]:
df['min_nights_bucket'] = pd.cut(df['minimum_nights'], 
                                  bins=[0,1,3,7,30,1000], 
                                  labels=['1', '2-3', '4-7', '8-30', '30+'])

min_nights_perf = df.groupby('min_nights_bucket')['number_of_reviews_ltm'].mean().reset_index()

In [ ]:
fig = px.bar(min_nights_perf, x='min_nights_bucket', y='number_of_reviews_ltm',
             title='2-3 Night Minimum = Highest Booking Volume')
fig.update_layout(height=400)
fig.show()

print(df.groupby('min_nights_bucket')['number_of_reviews_ltm'].mean().round(1))

In [ ]:
sample = df.sample(min(2000, len(df)), random_state=42)

fig = px.scatter_mapbox(sample, 
                        lat='latitude', lon='longitude',
                        color='min_nights_bucket',
                        color_discrete_map={'1': '#636EFA', '2-3': '#00CC96', '4-7': '#FFA15A', '8-30': '#EF553B', '30+': '#AB63FA'},
                        zoom=11,
                        hover_data=['neighbourhood_cleansed', 'room_type', 'price_clean', 'number_of_reviews_ltm', 'minimum_nights'],
                        title='Listing Locations by Minimum Nights')

fig.update_layout(mapbox_style='carto-positron', height=500)
fig.show()

In [ ]:
min_nights_by_hood = df.groupby('neighbourhood_cleansed')['minimum_nights'].median().sort_values(ascending=True)

fig = px.bar(x=min_nights_by_hood.values, 
             y=min_nights_by_hood.index,
             orientation='h',
             title='Median Minimum Nights by Neighbourhood',
             labels={'x': 'Median Minimum Nights', 'y': 'Neighbourhood'})

fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
fig.show()

**Observation:** 2-3 nights is optimal. 30+ nights kills bookings.

**Hypothesis:** Many hosts over-restrict, leaving money on table. (Maybe a different segment targeting different audience? Touristy centre tends to have more 2-night while outer segment longer minimum stays. Perhaps centre is more optimized?)

These findings suggest patterns worth investigating further. Key limitations:

 - Cross-sectional data (no trends)
 - Reviews as booking proxy (not actual bookings)
 - Correlation of platform feature states, not proven causation

Next steps would require calendar data, longitudinal tracking, or A/B testing to move from 'associated with' to 'causes.'